In [1]:
root_folder = "../results"

In [2]:
metrics2paper = {
    "R2": "R2 (val)",
    "MAPE": "MAPE (val)",
    "RMSE": "RMSE (val)",
    "NRMSE": "%RMSE (val)",
    "R2_train": "R2 (train)",
    "MAPE_train": "MAPE (train)",
    "RMSE_train": "RMSE (train)",
    "NRMSE_train": "%RMSE (train)",
}

In [3]:
import pandas as pd
from glob import glob
from experiments_to_run import *

paths = glob(f'{root_folder}/metrics*')

df = []
for path in paths:
    tmp_df = pd.read_pickle(path)
    df.append(tmp_df)
df = pd.concat(df)

2024-08-08 14:14:48.013577: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-08 14:14:48.130114: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-08 14:14:48.130132: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-08-08 14:14:48.718786: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [4]:
metrics = ["R2", "MAPE", "RMSE", "NRMSE", "R2_train", "MAPE_train", "RMSE_train", "NRMSE_train"]

df_ens = df[df.model == "▸ Ensemble"]
df = df[df.model != "▸ Ensemble"]

mean_df = df.groupby(["target", "config", "model"]).agg("mean")[metrics]
std_df = df.groupby(["target", "config", "model"]).agg("std")[metrics]

mean_df_ens = df_ens.groupby(["target", "config", "model"]).agg("mean")[metrics]
std_df_ens = df_ens.groupby(["target", "config", "model"]).agg("std")[metrics]

for target in mean_df.index.levels[0]:
    for config in mean_df.index.levels[1]:
        
        # add Average model performance
        item = pd.DataFrame(
            mean_df.loc[target, config][metrics].mean().to_dict(),
            index=[(target, config, "▸ Average")]
        )
        mean_df = pd.concat((mean_df, item))

for target in std_df.index.levels[0]:
    for config in std_df.index.levels[1]:
        
        # add Average model performance
        item = pd.DataFrame(
            std_df.loc[target, config][metrics].mean().to_dict(),
            index=[(target, config, "▸ Average")]
        )
        std_df = pd.concat((std_df, item))

mean_df = pd.concat([mean_df, mean_df_ens])
std_df = pd.concat([std_df, std_df_ens])


std_df.columns = ["s_"+x for x in std_df.columns]
overall_df = pd.concat((mean_df, std_df), axis=1)

for target in TARGETS:
    tmp = overall_df.reset_index()[(overall_df.reset_index().target==target)].drop(columns=["target"])
    print("results exported to", f"{root_folder}/overall_metrics--{target}.[pickle/csv]")
    tmp.to_pickle(f"{root_folder}/overall_metrics--{target}.pickle")
    tmp.to_csv(f"{root_folder}/overall_metrics--{target}.csv", index=False)


for metric in metrics:
    overall_df[f"str_{metric}"] = overall_df.apply(lambda row: f"{row[metric]:.2f} ± {row['s_'+metric]:.2f}", axis=1)
    
overall_df = overall_df.drop(columns = [x for x in overall_df.columns if "str" not in x])
overall_df.columns = metrics
overall_df = overall_df.reset_index()

overall_df.sort_values(["target","config", "model"], inplace=True)
overall_df.rename(columns=metrics2paper, inplace=True)
overall_df

/tmp/ipykernel_22268/630951020.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
  mean_df.loc[target, config][metrics].mean().to_dict(),
/tmp/ipykernel_22268/630951020.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
  mean_df.loc[target, config][metrics].mean().to_dict(),
/tmp/ipykernel_22268/630951020.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
  mean_df.loc[target, config][metrics].mean().to_dict(),
/tmp/ipykernel_22268/630951020.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
  mean_df.loc[target, config][metrics].mean().to_dict(),
/tmp/ipykernel_22268/630951020.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
  mean_df.loc[target, config][metrics].mean().to_dict(),
/tmp/ipykernel_22268/630951020.py:17: PerformanceWarning: indexing past lexsort depth may impact performance.
  mean_df.loc[target, config][metrics].mean().to_dict()

results exported to ../results/overall_metrics--CS.[pickle/csv]
results exported to ../results/overall_metrics--CSE.[pickle/csv]


,target,config,model,R2 (val),MAPE (val),RMSE (val),%RMSE (val),R2 (train),MAPE (train),RMSE (train),%RMSE (train)
0,CS,Conf1,AdaBoost,0.32 ± 0.09,2.50 ± 1.52,49.82 ± 11.06,65.08 ± 15.61,0.91 ± 0.02,1.46 ± 0.26,18.52 ± 1.32,24.10 ± 1.76
1,CS,Conf1,BaggedDT,0.31 ± 0.08,2.51 ± 1.72,50.11 ± 9.59,65.44 ± 13.82,0.90 ± 0.01,1.03 ± 0.18,19.73 ± 0.76,25.69 ± 1.29
2,CS,Conf1,BayesianNN,0.11 ± 0.10,3.95 ± 3.58,57.06 ± 12.68,74.58 ± 18.20,0.19 ± 0.04,3.84 ± 0.85,55.36 ± 3.64,72.01 ± 3.75
3,CS,Conf1,CatBoost,0.32 ± 0.07,2.60 ± 1.81,49.64 ± 9.95,64.86 ± 14.44,0.82 ± 0.01,1.43 ± 0.28,26.26 ± 1.27,34.18 ± 1.66
4,CS,Conf1,DeepCNN,0.23 ± 0.11,3.22 ± 2.87,53.04 ± 12.64,69.23 ± 18.19,0.38 ± 0.30,2.75 ± 1.47,47.72 ± 14.51,61.59 ± 17.37
...,...,...,...,...,...,...,...,...,...,...,...
101,CSE,Conf4,SVR,-0.04 ± 0.20,1.87 ± 0.97,1.19 ± 0.08,71.04 ± 6.95,0.40 ± 0.04,1.32 ± 0.23,0.93 ± 0.02,55.09 ± 1.27
102,CSE,Conf4,StackEns,0.37 ± 0.08,1.20 ± 0.29,0.93 ± 0.10,55.74 ± 7.44,0.80 ± 0.02,0.77 ± 0.08,0.54 ± 0.02,31.90 ± 1.72
103,CSE,Conf4,XGBoost,0.39 ± 0.13,1.19 ± 0.34,0.91 ± 0.11,54.33 ± 7.38,0.97 ± 0.01,0.33 ± 0.04,0.22 ± 0.02,13.03 ± 1.25
111,CSE,Conf4,▸ Average,0.27 ± 0.12,1.42 ± 0.48,0.99 ± 0.10,59.25 ± 7.15,0.65 ± 0.02,0.99 ± 0.14,0.65 ± 0.03,38.80 ± 1.61


In [5]:
for target in TARGETS:
    tmp = overall_df[(overall_df.target==target)].drop(columns=["target"])
    print("results exported to", f"{root_folder}/table_{target}.[txt/xlsx]")
    tmp.to_markdown(f"{root_folder}/table_{target}.txt", index=False, tablefmt="fancy_grid")
    tmp.to_excel(f"{root_folder}/table_{target}.xlsx", index=False)
    
    print("results exported to", f"../figures_and_tables/table_performance_{target}.[txt/xlsx/csv]")
    tmp.to_markdown(f"../figures_and_tables/table_performance_{target}.txt", index=False, tablefmt="fancy_grid")
    tmp.to_excel(f"../figures_and_tables/table_performance_{target}.xlsx", index=False)
    tmp.to_csv(f"../figures_and_tables/table_performance_{target}.csv", index=False)

results exported to ../results/table_CS.[txt/xlsx]
results exported to ../figures_and_tables/table_performance_CS.[txt/xlsx/csv]
results exported to ../results/table_CSE.[txt/xlsx]
results exported to ../figures_and_tables/table_performance_CSE.[txt/xlsx/csv]
